In [1]:
import pandas as pd

In [2]:
events = pd.DataFrame.from_csv('course-217-events.csv', index_col=None)
events.sort_values('time', inplace=True)
events.head()

,user_id,action,step_id,time
1431922,1229,viewed,38518,1440065573
1431921,1229,discovered,38518,1440065573
1431920,1229,passed,38518,1440065573
1431917,1229,passed,38521,1440065578
1431919,1229,viewed,38521,1440065578


In [3]:
structure = pd.read_csv('course-217-structure.csv', index_col='step_id')
structure.head()

,course_id,module_id,module_position,lesson_id,lesson_position,step_position,step_type,step_cost
step_id,,,,,,,,
38842,217,614,2,13228,2,1,text,0
39715,217,614,2,13228,2,6,code,1
39716,217,614,2,13228,2,7,code,1
39717,217,614,2,13228,2,8,code,1
39721,217,614,2,13228,2,9,free-answer,0


In [4]:
from collections import defaultdict

In [5]:
step_positions = dict()
for i, row in structure.iterrows():
    step_positions[i] = (row.module_position, row.lesson_position, row.step_position)

In [6]:
step_positions_T = {v: k for k, v in step_positions.items()}

In [7]:
from progressbar import ProgressBar

In [21]:
who_watched = defaultdict(set)
who_rewatched = defaultdict(set)

def get_next_step(x, y, z):
    if (x, y, z + 1) in step_positions_T:
        return step_positions_T[(x, y, z + 1)]
    elif (x, y + 1, 1) in step_positions_T:
        return step_positions_T[(x, y + 1, 1)]
    elif (x + 1, 1, 1) in step_positions_T:
        return step_positions_T[(x + 1, 1, 1)]
    else:
        return None
    
bar = ProgressBar(max_value=len(events.user_id.unique()))

for user_id, user_events in bar(events.groupby('user_id')):
    for i, row in user_events.iterrows():
        if user_id in who_watched[row.step_id]:
            x, y, z = step_positions[row.step_id]
            next_step_id = get_next_step(x, y, z)
            if user_id in who_watched[next_step_id]:
                _t = (user_events.time < row.time)
                cur = user_events[_t & (user_events.step_id == row.step_id)]
                nex = user_events[_t & (user_events.step_id == next_step_id)]
                if len(cur) != 0 and (len(nex) == 0 or cur.iloc[0].time < nex.iloc[-1].time):
                    who_rewatched[row.step_id].add(user_id)
        else:
            who_watched[row.step_id].add(user_id)

100% (7662 of 7662) |#####################| Elapsed Time: 0:28:00 Time: 0:28:00


In [22]:
for k, v in list(who_watched.items()):
    if len(v) == 0:
        del who_watched[k]

In [23]:
for k, v in list(who_rewatched.items()):
    if len(v) == 0:
        del who_rewatched[k]

In [24]:
all(len(v) > 0 for k, v in who_rewatched.items())

True

In [25]:
step_rewatch_rates = dict()
for step_id in who_watched:
    step_rewatch_rates[step_id] = len(who_rewatched[step_id]) / len(who_watched[step_id])

In [26]:
best_steps = sorted(step_rewatch_rates.items(), key=lambda it: it[1], reverse=True)[:10]
best_steps

[(41684, 0.7460106382978723),
 (41604, 0.7297605473204105),
 (41097, 0.6809314033983638),
 (41481, 0.6748329621380846),
 (42593, 0.6662468513853904),
 (38872, 0.6600851722802943),
 (41686, 0.6571815718157181),
 (39735, 0.6543824701195219),
 (39740, 0.6538882803943045),
 (39717, 0.6459863945578231)]

In [27]:
for k, v in best_steps:
    print(k, end=',')

41684,41604,41097,41481,42593,38872,41686,39735,39740,39717,